Axioms for the theory of <a class="ProveItLink" href="theory.ipynb">proveit.physics.quantum</a>
========

In [ ]:
import proveit
# Prepare this notebook for defining the axioms of a theory:
%axioms_notebook # Keep this at the top following 'import proveit'.
from proveit import (Operation, Function, Lambda, Conditional,
                     ExprArray, ExprTuple, Composition)
from proveit import (a, b, c, f, k, m, n, v, x, y, alpha, 
                     A, B, M, U, V, W, X, Y, Z, fx, fy)
from proveit.core_expr_types import A_1_to_m, B_1_to_m
from proveit.linear_algebra import (
    LinMap, VecSpaces, MatrixSpace, MatrixMult, TensorProd, SU, ScalarMult)
from proveit.logic import (Iff, And, Or, Forall, Exists, Equals, 
                           InSet, Set, Union, SetOfAll, InClass)
from proveit.numbers import zero, one, two, frac, Integer, Natural, NaturalPos, Complex, Exp
from proveit.numbers import Add, Exp, Mult, sqrt, subtract, greater
from proveit.numbers.number_sets import Interval
from proveit.physics.quantum import (
    QmultCodomain, Hspace, QuantumMult,
    Ket, Input, Output, RegisterKet)
from proveit.physics.quantum.circuit import Gate, MultiQubitGate, CircuitEquiv, Circuit, MultiWire
# from proveit.physics.quantum import Bra, Ket, RegisterBra, Meas, MultiWire, Circuit
from proveit.physics.quantum import ket0, ket1, ket_plus, H, QubitSpace, RegisterSU, I, CONTROL
# from proveit.physics.quantum import I, X, Y, Z, RegisterSU
from proveit.physics.quantum import QubitRegisterSpace

In [ ]:
%begin axioms

In [ ]:
ket_zero_in_qubit_space = InSet(ket0, QubitSpace)

In [ ]:
ket_one_in_qubit_space = InSet(ket1, QubitSpace)

In [ ]:
single_qubit_register_ket = Forall(
        x,
        Equals(RegisterKet(x, one), Ket(x)))

In [ ]:
single_qubit_register_zero = Equals(RegisterKet(zero, one), ket0)

In [ ]:
single_qubit_register_one = Equals(RegisterKet(one, one), ket1)

### QuantumMult and QuantumMultCodomain

A QuantumMult Operation can string together a sequences of quantum operators and/or kets.  Properly defined, a ket is a vector in a Hilbert space and a quantum operator acts (under QuantumMult) as a linear map from a Hilbert space to a Hilbert space or a complex number.  The latter is called a bra.

The result of a QuantumMult Operation, if and only if the they are in a valid sequency, will be in the QmultCodomain class which includes all vectors in any vector space over complex numbers or linear maps between vectors of vector spaces over complex numbers:

In [ ]:
Forall(x, Equals(InClass(x, QmultCodomain),
                 Or(Exists(Hspace, InSet(x, Hspace),
                           domain=VecSpaces(Complex)),
                    Exists((Hspace, X), InSet(x, LinMap(Hspace, X)),
                           domain=VecSpaces(Complex)))
                .with_wrap_after_operator()))

When a QuantumMult has a single quantum operator or bra operand, it represents the corresponding linear mapping of the operand and we format it as the operand wrapped in square brackets.

The corresponding linear map for the matrix performs the matrix multiplication:

In [ ]:
qmult_of_matrix = Forall(
    (m), Forall(M, Equals(QuantumMult(M), 
                             Lambda(x, MatrixMult(M, x))),
                   domain=MatrixSpace(Complex, m, m)),
    domain=NaturalPos)

The corresponding linear map for a linear map is the linear map itself:

In [ ]:
qmult_of_linmap = Forall(
    (Hspace, X), Forall(A, Equals(QuantumMult(A), A),
                        domain=LinMap(Hspace, X)),
    domain=VecSpaces(Complex))

A QuantumMult of a bra or operator applied to a ket is the application of the corresponding mapping (from Hilbert space to a c-number or Hilbert space to Hilbert space):

In [ ]:
qmult_op_ket = Forall(
    (Hspace, X), Forall(
        A, Forall(k, Equals(QuantumMult(A, k),
                            Function(QuantumMult(A), k)),
                  domain=Hspace),
        condition=InSet(QuantumMult(A),
                        LinMap(Hspace, X))),
    domain=VecSpaces(Complex))

We define the QuantumMult of two operators as the composition of the operators:

In [ ]:
qmult_op_op = Forall(
    (Hspace, X, Y), Forall(
        (A, B), Equals(QuantumMult(A, B),
                       Composition(QuantumMult(A),
                                   QuantumMult(B))),
        conditions=[InSet(QuantumMult(A), LinMap(X, Y)),
                    InSet(QuantumMult(B), LinMap(Hspace, X))]),
    domain=VecSpaces(Complex))

We define the QuantumMult of ket followed by a bra as the outer product mapping which is consistent with QuantumMult associativity:

In [ ]:
qmult_ket_bra = Forall(
    (Hspace, X), Forall(
        k, Forall(
            A, Equals(QuantumMult(k, A),
                      Lambda(v, Conditional(
                          ScalarMult(QuantumMult(A, v), k),
                          InSet(v, Hspace)))),
            condition=InSet(QuantumMult(A), 
                            LinMap(Hspace, Complex))),
        domain=X))

We define the QuantumMult of a c-number with anything, with the c-number on either side, as the scalar multiplication of the c number with the other operand:

In [ ]:
Forall((a, b), Equals(QuantumMult(a, b),
                      Mult(a, b)),
       domain=Complex)

In [ ]:
Forall(c, Forall(X, Equals(QuantumMult(c, X),
                           ScalarMult(c, X)),
                 domain=QmultCodomain),
       domain=Complex)

In [ ]:
Forall(c, Forall(X, Equals(QuantumMult(X, c),
                           ScalarMult(c, X)),
                 domain=QmultCodomain),
       domain=Complex)

By the following axiom, we can know that a QuantumMult only results in an element of its codomain if the operands are in a valid sequence (op-ket, op-op, ket-bra, complex-any, or any-complex):

In [ ]:
qmult_in_codomain_only_if_valid = Forall(
        (A, B), Exists(
            (Hspace, X, Y),
            InSet(ExprTuple(QuantumMult(A), QuantumMult(B)),
                  Set(ExprTuple(LinMap(Hspace, X), Hspace),
                      ExprTuple(X, LinMap(Hspace, Complex)),
                      ExprTuple(LinMap(X, Y), LinMap(Hspace, X)),
                      ExprTuple(Complex, QmultCodomain),
                      ExprTuple(QmultCodomain, Complex))),
            domain=VecSpaces(Complex)),
        condition=InClass(QuantumMult(A, B),
                          QmultCodomain))

In [ ]:
multi_quantum_mult_def = \
    Forall(m, Forall((A_1_to_m, B), 
                     Equals(QuantumMult(A_1_to_m, B), 
                            QuantumMult(QuantumMult(A_1_to_m), B))
                     .with_wrap_after_operator()),
           domain=Natural)

By definition, a bra or quantum operator transforms to the corresponding linear map when wrapped in QuantumOpMult.

Don't regard trivial 1-column matrics (which could be regarded as the same as a column-vector) as quantum operators.

In [ ]:
scalar_id_for_ket = Forall(k,
                           Equals(Mult(one, Ket(k)), Ket(k)),
                           domain=Natural)

In [ ]:
expand_register_with_zero_ket = Forall(
        n,
        Forall(k,
               Equals(RegisterKet(k, Add(n, one)),
                      TensorProd(ket0, RegisterKet(k, n))),
               domain=Interval(zero, subtract(Exp(two, n), one))),
        domain=NaturalPos)

In [ ]:
expand_register_with_one_ket = Forall(
        n, 
        Forall(k, Equals(RegisterKet(Add(k, Exp(two, n)), Add(n, one)),
                         TensorProd(ket1, RegisterKet(k, n))),
               domain=Interval(zero, subtract(Exp(two, n), one))),
        domain=NaturalPos)

In [ ]:
ket_plus_def = Equals(ket_plus, frac(Add(ket0, ket1), sqrt(two)))

In [ ]:
hadamard_on_zero = Equals(MatrixMult(H, ket0), ket_plus)

In [ ]:
empty_gate = Equals(Gate(), I.with_styles(representation="explicit")) # base case

In [ ]:
substitution = Forall((f, x, y), CircuitEquiv(fx, fy), conditions=CircuitEquiv(x, y))

And then we have several axioms involving the Circuit class, which class itself still needs updating<br/>(in particular, the Circuit class needs an appropriate substitute for the ExpressionTensor class -- perhaps ExprArray).

In [ ]:
circuit_gate_application = Forall(
    U, 
    Forall((x, y),
           Iff(Circuit(ExprArray(ExprTuple(Input(x), Gate(U), Output(y)))),
               Equals(y, MatrixMult(U, x))),
           domain=QubitSpace),
    domain=SU(two))

In [ ]:
circuit_multi_gate_application = Forall(
    n,
    Forall(U,
           Forall((x, y),
                  Iff(Circuit(ExprArray(ExprTuple(Input(x), MultiWire(n), Gate(U), Output(y)))),
                      Equals(y, MatrixMult(U, x))),
                  domain=QubitRegisterSpace(n)),
           domain=RegisterSU(n)),
    domain=NaturalPos)

In [ ]:
zero_controlled_ngate = Forall(
    n,
    Forall(U,
           Forall(x,
                  Forall(alpha,
                         Circuit(ExprArray(ExprTuple(Input(ScalarMult(alpha, ket0)),
                                   I,
                                   MultiQubitGate(CONTROL, Set(one, two)),
                                   Output(ScalarMult(alpha, ket0))),
                                  ExprTuple(Input(x), MultiWire(n), MultiQubitGate(U, Set(one, two)), Output(x)))),
                         domain=Complex),
                  domain=QubitRegisterSpace(n)),
           domain=SU(Exp(two, n))),
    domain=NaturalPos)

In [ ]:
one_controlled_ngate = Forall(
    n,
    Forall(U,
           Forall(x,
                  Forall(alpha,
                         Circuit(ExprArray(ExprTuple(Input(ScalarMult(alpha, ket1)),
                                   I,
                                   MultiQubitGate(CONTROL, Set(one, two)),
                                   Output(ScalarMult(alpha, ket1))),
                                  ExprTuple(Input(x), MultiWire(n), MultiQubitGate(U, Set(one, two)), Output(MatrixMult(U, x))))),
                         domain=Complex),
                  domain=QubitRegisterSpace(n)),
           domain=SU(Exp(two, n))),
    domain=NaturalPos)

In [ ]:
%end axioms